In [1]:
# Sort based on match scores, 
# 1. cutoff, 2. save file with annotation
import collections

import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)
one_recom = recom.getSpeciesRecommendation(pred_id='SAM')
two_recom = recom.getReactionRecommendation(pred_id='VCoA')

In [3]:
filt_recom = recom.filterRecommendationByThreshold(rec=one_recom,
                                                   thresh=0.8)
filt_recom.candidates

[('CHEBI:15414', 1.0), ('CHEBI:67040', 1.0)]

In [4]:
filt_recom.candidates

[('CHEBI:15414', 1.0), ('CHEBI:67040', 1.0)]

In [5]:
from AMAS import annotation_maker as am

In [6]:
maker = am.AnnotationMaker('species')

In [7]:
container_items = ['annotation', 
                   am.RDF_TAG,
                   'rdf:Description rdf:about="#'+'meta_id'+'"',
                   maker.prefix,
                   'rdf:Bag']

In [10]:
model = recom.sbml_document.getModel()
one_s = model.getSpecies('SAM')
exist_anot_str = one_s.getAnnotationString()
print(exist_anot_str)

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [12]:
exist_anot_list = exist_anot_str.split('\n')
exist_anot_list

['<annotation>',
 '  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">',
 '    <rdf:Description rdf:about="#metaid_0000036">',
 '      <bqbiol:is>',
 '        <rdf:Bag>',
 '          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>',
 '          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>',
 '        </rdf:Bag>',
 '      </bqbiol:is>',
 '    </rdf:Description>',
 '  </rdf:RDF>',
 '</annotation>']

In [101]:
ELEMENT = 'species'
def divideExistingAnnotation(inp_str):
  """
  Divide existing string annotation
  into an empty container and
  items; 
  
  Parameters
  ----------
  inp_str: str
  
  Returns
  -------
  :dict
      Dictionary of container,
      and items to be augmented
  """
  exist_anot_list = inp_str.split('\n')
  template_container = []
  items = []
  one_line = ''
  while one_line.strip() != '<rdf:Bag>' and exist_anot_list:
    one_line = exist_anot_list.pop(0)
    template_container.append(one_line)

  one_line = exist_anot_list.pop(0)
  while one_line.strip() != '</rdf:Bag>' and exist_anot_list:
    items.append(one_line.strip())
    one_line = exist_anot_list.pop(0)

  template_container.append(one_line)
  while exist_anot_list:
    one_line = exist_anot_list.pop(0)
    template_container.append(one_line)  
  res = {'container': template_container,
         'items': items}
  return res

def augmentAnnotation(candidates,
                      annotation):
  """
  Augment existing annotations
  (meta id is supposed to be included
  in the existing annotation)
  
  Parameters
  ----------
  candidates: str-list
      List of candidates
      
  existing_annotation: str
      Existing element annotation
  """
  annotation_dict = divideExistingAnnotation(annotation)
  container = annotation_dict['container']
  existing_items = annotation_dict['items']
  existing_identifiers = []
  for val in existing_items:
    url = re.findall('"(.*?)"', val)[0]
    existing_identifiers.append(url.split('/')[-1])
  additional_identifiers = [val for val in candidates \
                            if val not in existing_identifiers]
  new_items = [maker.createAnnotationItem(am.KNOWLEDGE_RESOURCE[ELEMENT],one_cand) \
               for one_cand in additional_identifiers]
  items = existing_items + new_items
  res = maker.insertList(template_container, items)
  return '\n'.join(res)

In [104]:
print(exist_anot_str)

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [102]:
print(augmentAnnotation(['CHEBI:00000'], exist_anot_str))

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:00000"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [97]:
exist_anot_str

'<annotation>\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">\n    <rdf:Description rdf:about="#metaid_0000036">\n      <bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>\n          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>\n        </rdf:Bag>\n      </bqbiol:is>\n    </rdf:Description>\n  </rdf:RDF>\n</annotation>'

In [93]:
import re

existing_identifiers = []
for val in res['items']:
  url = re.findall('"(.*?)"', val)[0]
  existing_identifiers.append(url.split('/')[-1])

In [94]:
existing_identifiers

['CHEBI:15414', 'C00019']

In [64]:
exist_anot_list = exist_anot_str.split('\n')

template_container = []
items_tobe_added = []
one_line = ''
while one_line.strip() != '<rdf:Bag>' and exist_anot_list:
  one_line = exist_anot_list.pop(0)
  template_container.append(one_line)
    
one_line = exist_anot_list.pop(0)
while one_line.strip() != '</rdf:Bag>' and exist_anot_list:
  items_tobe_added.append(one_line.strip())
  one_line = exist_anot_list.pop(0)

template_container.append(one_line)
while exist_anot_list:
  one_line = exist_anot_list.pop(0)
  template_container.append(one_line)  

res = {'container': template_container,
       'items': items_tobe_added}

In [59]:
template_container

['<annotation>',
 '  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">',
 '    <rdf:Description rdf:about="#metaid_0000036">',
 '      <bqbiol:is>',
 '        <rdf:Bag>',
 '        </rdf:Bag>',
 '      </bqbiol:is>',
 '    </rdf:Description>',
 '  </rdf:RDF>',
 '</annotation>']

In [60]:
items_tobe_added

['<rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>',
 '<rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>']

In [61]:
maker.insertList(template_container, items_tobe_added)

['<annotation>',
 '  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">',
 '    <rdf:Description rdf:about="#metaid_0000036">',
 '      <bqbiol:is>',
 '        <rdf:Bag>',
 '          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>',
 '          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>',
 '        </rdf:Bag>',
 '      </bqbiol:is>',
 '    </rdf:Description>',
 '  </rdf:RDF>',
 '</annotation>']

In [14]:
container_items = ['annotation', 
                   'rdf:Description rdf:about="#',
                   'rdf:Bag']
maker.createAnnotationContainer(container_items)

['<annotation>',
 '  <rdf:Description rdf:about="#>',
 '    <rdf:Bag>',
 '    </rdf:Bag>',
 '  </rdf:Description>',
 '</annotation>']

In [8]:
maker.createAnnotationContainer(container_items)

['<annotation>',
 '  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">',
 '    <rdf:Description rdf:about="#meta_id">',
 '      <bqbiol:is>',
 '        <rdf:Bag>',
 '        </rdf:Bag>',
 '      </bqbiol:is>',
 '    </rdf:Description>',
 '  </rdf:RDF>',
 '</annotation>']

In [9]:
candidates = ['CHEBI:15414', 'CHEBI:67040']
res = maker.getAnnotationString(candidates, 'meta_id_00111')

In [10]:
print(res)

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#meta_id_00111">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:67040"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [11]:
model = recom.sbml_document.getModel()
one_s = model.getSpecies('SAM')
print(one_s.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/obo.chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.compound/C00019"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [13]:
one_s.setAnnotation(res)
print(one_s.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000036">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:67040"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [14]:
maker.createAnnotationContainer(['annotation'])

['<annotation>', '</annotation>']

In [20]:
ONE_CHEBI = 'CHEBI:15414'
ONE_CHEBI_SCORE = 1.00
TWO_CHEBI = 'CHEBI:59789'
TWO_CHEBI_SCORE = 1.00
CANDIDATES = [ONE_CHEBI, TWO_CHEBI]
CHEBI = 'chebi'

ONE_ANNOTATION_ITEM = '<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>'
ONE_SCORE_ITEM = '<rdf:li rdf:resource="http://reproduciblebiomodels.org/amas/v1/by_name/0.20"/>'
ONE_CONTAINER = ['<annotation>', '</annotation>']
TWO_CONTAINER = ['<rdf:RDF>', '</rdf:RDF>']

with open(os.path.join(cn.TEST_DIR, 'full_annotation_example.txt')) as file:
    lines = [line.rstrip() for line in file]
FULL_ANNOTATION = '\n'.join(lines)

ONE_INSERTED = ['<annotation>', '  <rdf:RDF>', '  </rdf:RDF>', '</annotation>']
TWO_INSERTED = ['rdf:RDF']

In [21]:
maker.createAnnotationItem(CHEBI,
                                               ONE_CHEBI)

'<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>'

In [23]:
print(maker.getAnnotationString(CANDIDATES, meta_id='metaid_00001'))

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vcard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_00001">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15414"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:59789"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>
